In [4]:
import warnings
warnings.filterwarnings("ignore")
# !pip install ftfy
import ftfy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
from math import exp
from numpy import sign
import sys
print(sys.executable)
from PIL import Image # getting images in notebook
# !pip install gensim
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# !pip install tensorflow

# !pip install tensorflow_hub

# !pip install bert-for-tf2
# !pip install sentencepiece

C:\Users\SHWETANK VERMA\Anaconda3\latest\python.exe


[nltk_data] Downloading package stopwords to C:\Users\SHWETANK
[nltk_data]     VERMA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\SHWETANK
[nltk_data]     VERMA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# from tensorflow import keras 
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
import re
from tqdm import tqdm
import tensorflow as tf
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [6]:
train=pd.read_csv(r'C:\Users\SHWETANK VERMA\Documents\Mlstuff\Major-1\Datasets\train.csv')

In [7]:
train.sample(3)

,PID,Text_data,Label
7934,train_pid_7935,"I have panic attacks a lot, like once or twice...",not depression
7123,train_pid_7124,Awake for 15 hours before I talked to another ...,not depression
5102,train_pid_5103,"New Year, Same Loneliness : Not like I hoped t...",moderate


In [8]:
test=pd.read_csv(r'C:\Users\SHWETANK VERMA\Documents\Mlstuff\Major-1\Datasets\dev.csv')

In [9]:
print("Training set has {} rows and {} columns.".format(train.shape[0], train.shape[1]))
print("Test set has {} rows and {} columns.".format(test.shape[0], test.shape[1]))

print()
print(train.columns)
print(test.columns)

Training set has 8891 rows and 3 columns.
Test set has 4496 rows and 3 columns.

Index(['PID', 'Text_data', 'Label'], dtype='object')
Index(['PID', 'Text data', 'Label'], dtype='object')


In [10]:
Sentiment = {'moderate': 1,'not depression': 0,'severe':2}
train.Label = [Sentiment[item] for item in train.Label]
test.Label= [Sentiment[item] for item in test.Label]

In [11]:
train.sample(3)

,PID,Text_data,Label
5352,train_pid_5353,"Can I take a moment just to rant? : So, my gf’...",1
6219,train_pid_6220,I want to help my very depressed friend but id...,0
4397,train_pid_4398,part of me won’t leave : i try to better mysel...,1


In [12]:
test.sample(3)


,PID,Text data,Label
1336,dev_pid_1337,I’ve been in quarantine in my head for 2 years...,1
2033,dev_pid_2034,How bad is bad enough? : Hi :) \nThis is my fi...,1
2759,dev_pid_2760,Had a huge fight with my parents- blamed them ...,0


In [13]:
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# NLTK Tweet Tokenizer for now
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)

corpus = []

# clean up text
def clean_text(text):
    """
    Copied from other notebooks
    """
    # expand acronyms
    
    # special characters
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    
    # emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    
    """
    Our Stuff
    """
    # remove numbers
    text = re.sub(r'[0-9]', '', text)
    
    # remove punctuation and special chars (keep '!')
    for p in string.punctuation.replace('!', ''):
        text = text.replace(p, '')
        
    # remove urls
    text = re.sub(r'http\S+', '', text)
    
    # tokenize
    text = tknzr.tokenize(text)
    
    # remove stopwords
    text = [w.lower() for w in text if not w in stop_words]
    corpus.append(text)
    
    # join back
    text = ' '.join(text)
    
    return text

In [14]:
%%time
train['Text_data'] = train['Text_data'].apply(lambda s: clean_text(s))
test['Text data'] = test['Text data'].apply(lambda s: clean_text(s))

# see some cleaned data
train.sample(10)

Wall time: 12.2 s


,PID,Text_data,Label
1394,train_pid_1395,i want stop removed,1
702,train_pid_703,alone nye another holiday im tired alone time ...,1
8091,train_pid_8092,i ’ want live life sick tired i chronic painfa...,2
2758,train_pid_2759,feeling alone holiday i spend next contact fam...,1
4586,train_pid_4587,i tried cut couldnt i need someone talk,1
7008,train_pid_7009,i ’ proud alive making this year tough i ’ sur...,0
6510,train_pid_6511,imaa fishhhh removed,0
3271,train_pid_3272,those learned feel happiness life pointless wi...,1
4221,train_pid_4222,everyday physical pain i chronic pain conditio...,1
6459,train_pid_6460,happy new year i started sending friends text ...,0


In [15]:
!pip install spacy-download
!pip install allennlp
# python -m spacy download en_core_web_sm
# import spacy
# print(spacy.__file__)
# import sys
# print(sys.executable)

In [16]:
# !pip install allennlp
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Tokenize the text using spaCy
    doc = nlp(text)
    # Remove stop words and lemmatize the tokens
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.is_alpha]
    # Join the tokens back into a string
    return " ".join(tokens)


In [17]:
# Preprocess the training and testing text data
train["Text_data"] = train["Text_data"].apply(preprocess_text)
test["Text data"] = test["Text data"].apply(preprocess_text)

In [18]:
train["Text_data"][0]

'wait mind breakdown new year feeling anymore know little bit worried depressed day time year try breakdown start mere day later break cry entire year december ok month wait weird way act feel feel bit normal'

In [19]:
def extractF(df):
    verbs = []
    args = []

    for text in df["Text_data"]:
        doc = nlp(text)
        for token in doc:
            if token.pos_ == "VERB":
                verbs.append(token.text)
                for child in token.children:
                    if child.pos_ != "PUNCT":
                        args.append(child.text)

    # Create a vocabulary of unique verbs and arguments
    vocab = list(set(verbs + args))
    
    for verb in verbs:
        df[verb] = df["Text_data"].apply(lambda x: 1 if verb in x else 0)

    for arg in args:
        df[arg] = df["Text_data"].apply(lambda x: 1 if arg in x else 0)
    
    return vocab

In [20]:
def extractF2(df):
    verbs = []
    args = []

    for text in df["Text data"]:
        doc = nlp(text)
        for token in doc:
            if token.pos_ == "VERB":
                verbs.append(token.text)
                for child in token.children:
                    if child.pos_ != "PUNCT":
                        args.append(child.text)

    # Create a vocabulary of unique verbs and arguments
    vocab = list(set(verbs + args))
    
    for verb in verbs:
        df[verb] = df["Text data"].apply(lambda x: 1 if verb in x else 0)

    for arg in args:
        df[arg] = df["Text data"].apply(lambda x: 1 if arg in x else 0)
    
    return vocab

In [21]:
vocab1 = extractF(train)
vocab2 = extractF2(test)

In [22]:
vocab = list(set(vocab1 + vocab2))
vocab

['skater',
 'blow',
 'testosterone',
 'teleport',
 'income',
 'multitude',
 'connect',
 'dissipate',
 'doing',
 'depress',
 'grow',
 'cureall',
 'waitlist',
 'agonisingly',
 'crown',
 'perception',
 'legitimately',
 'childhood',
 'skull',
 'becuse',
 'recognize',
 'having',
 'roll',
 'enrol',
 'medicated',
 'seeked',
 'fruitlessly',
 'remorse',
 'praise',
 'reconnect',
 'unbelievable',
 'awakening',
 'welland',
 'sunway',
 'render',
 'increasingly',
 'pizza',
 'happend',
 'selfisolate',
 'workmate',
 'computing',
 'teaching',
 'harmless',
 'mitigated',
 'whouldnt',
 'diaper',
 'foreverfuck',
 'msg',
 'stay',
 'eyelash',
 'reamber',
 'accident',
 'plllease',
 'fro',
 'assault',
 'lining',
 'happy',
 'romantically',
 'everlasting',
 'probaly',
 'imagery',
 'will',
 'activate',
 'couple',
 'midst',
 'preinstalle',
 'questioning',
 'clause',
 'breakup',
 'swiftly',
 'curl',
 'unsurprisingly',
 'cafe',
 'bryant',
 'amid',
 'automationrobot',
 'energy',
 'hopes',
 'cause',
 'lessprivileged',

In [23]:
texts = train['Text_data'].to_numpy()
word_freq = {}

for text in texts:
    for word in text.split():
        word_freq[word] = word_freq.get(word, 0) + 1

In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_SEQUENCE_LENGTH = 40

tokenizer = Tokenizer()

tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
num_words = len(word_index) + 1
print('Found %s unique tokens.' % (num_words - 1))

# pad 
data = pad_sequences(
    sequences, 
    maxlen=MAX_SEQUENCE_LENGTH,
    padding='post', 
    truncating='post'
)

labels = train['Label'].to_numpy()
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 10272 unique tokens.
Shape of data tensor: (8891, 40)
Shape of label tensor: (8891,)


In [25]:
x_train = data
y_train = labels

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.metrics import roc_auc_score

vectorizer = CountVectorizer(vocabulary=vocab)
x_train_vectorized = vectorizer.fit_transform(train['Text_data'])

# print vocabulary
print(vectorizer.get_feature_names()[2500:2600])

['borderline', 'whoa', 'influence', 'flirt', 'remark', 'hollowed', 'hungry', 'unplanned', 'speaker', 'shift', 'kg', 'entwine', 'cali', 'layoff', 'pro', 'voice', 'morose', 'continously', 'construct', 'tongue', 'guilt', 'rewarding', 'routine', 'derealization', 'send', 'dissappeare', 'altercation', 'try', 'confession', 'scary', 'provoke', 'drift', 'craigslist', 'competitive', 'portrait', 'pay', 'ventrant', 'comedy', 'size', 'offender', 'rethink', 'fairytale', 'trust', 'longterm', 'deppresse', 'nocall', 'bing', 'babysit', 'underestimate', 'populated', 'healthcare', 'jjst', 'gig', 'holidays', 'exceed', 'immerse', 'dmv', 'admit', 'staff', 'london', 'thanks', 'lonleny', 'enter', 'certainty', 'lung', 'privilege', 'unfiltered', 'part', 'investigate', 'accept', 'heavy', 'complacency', 'mercy', 'likelyhood', 'panicing', 'ignorant', 'existto', 'pessimisticarguably', 'predispose', 'forsee', 'genuinely', 'statementand', 'kiddo', 'againwhatever', 'lovejust', 'talkspace', 'expect', 'we', 'breakdown', 

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [29]:
x = train["Text_data"]
y = train["Label"]

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [32]:
def prediction(X_test, model_object):
  
    # Predicton on test with giniIndex
    y_pred = model_object.predict(xv_test)
    print("Predicted values:")
    print(y_pred)
    return y_pred

In [33]:
def cal_accuracy(y_test, y_pred):
      
    print("Confusion Matrix: ",
        confusion_matrix(y_test, y_pred))
      
    print ("Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

In [34]:
# Decision tree with gini
model_gini = DecisionTreeClassifier(criterion = "gini",
            random_state = 123,max_depth=10, min_samples_leaf=6)
  
# Performing training
model_gini.fit(xv_train, y_train)

# Prediction using gini
y_pred_gini = prediction(xv_test, model_gini)
cal_accuracy(y_test, y_pred_gini)
print(classification_report(y_test,y_pred_gini))

Predicted values:
[1 1 1 ... 1 1 1]
Confusion Matrix:  [[ 195  302    6]
 [  63 1425   14]
 [   8  140   70]]
Accuracy :  76.0233918128655
              precision    recall  f1-score   support

           0       0.73      0.39      0.51       503
           1       0.76      0.95      0.85      1502
           2       0.78      0.32      0.45       218

    accuracy                           0.76      2223
   macro avg       0.76      0.55      0.60      2223
weighted avg       0.76      0.76      0.73      2223



In [35]:
# Decision tree with entropy
model_entropy = DecisionTreeClassifier(
            criterion = "entropy", random_state = 123,
            max_depth = 10, min_samples_leaf = 6)
  
# Performing training
model_entropy.fit(xv_train, y_train)

# Prediction using entropy
y_pred_entropy = prediction(xv_test, model_entropy)
cal_accuracy(y_test, y_pred_entropy)

print(classification_report(y_test,y_pred_entropy))

Predicted values:
[0 1 1 ... 1 2 1]
Confusion Matrix:  [[ 116  377   10]
 [   9 1482   11]
 [   5  101  112]]
Accuracy :  76.92307692307693
              precision    recall  f1-score   support

           0       0.89      0.23      0.37       503
           1       0.76      0.99      0.86      1502
           2       0.84      0.51      0.64       218

    accuracy                           0.77      2223
   macro avg       0.83      0.58      0.62      2223
weighted avg       0.80      0.77      0.72      2223



In [36]:
# using count vectorizer

In [39]:
vectorizer = CountVectorizer(vocabulary=vocab)

In [40]:

xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)

In [41]:
# Decision tree with gini
model_gini = DecisionTreeClassifier(criterion = "gini",
            random_state = 123,max_depth=10, min_samples_leaf=6)
  
# Performing training
model_gini.fit(xv_train, y_train)

# Prediction using gini
y_pred_gini = prediction(xv_test, model_gini)
cal_accuracy(y_test, y_pred_gini)
print(classification_report(y_test,y_pred_gini))

Predicted values:
[0 1 1 ... 1 1 1]
Confusion Matrix:  [[ 167  333    3]
 [  53 1446    3]
 [  12  144   62]]
Accuracy :  75.34862798020693
              precision    recall  f1-score   support

           0       0.72      0.33      0.45       503
           1       0.75      0.96      0.84      1502
           2       0.91      0.28      0.43       218

    accuracy                           0.75      2223
   macro avg       0.79      0.53      0.58      2223
weighted avg       0.76      0.75      0.72      2223



In [42]:
# Decision tree with entropy
model_entropy = DecisionTreeClassifier(
            criterion = "entropy", random_state = 123,
            max_depth = 10, min_samples_leaf = 6)
  
# Performing training
model_entropy.fit(xv_train, y_train)

# Prediction using entropy
y_pred_entropy = prediction(xv_test, model_entropy)
cal_accuracy(y_test, y_pred_entropy)

print(classification_report(y_test,y_pred_entropy))

Predicted values:
[0 1 1 ... 1 1 1]
Confusion Matrix:  [[ 113  384    6]
 [   3 1490    9]
 [   1  110  107]]
Accuracy :  76.92307692307693
              precision    recall  f1-score   support

           0       0.97      0.22      0.36       503
           1       0.75      0.99      0.85      1502
           2       0.88      0.49      0.63       218

    accuracy                           0.77      2223
   macro avg       0.86      0.57      0.62      2223
weighted avg       0.81      0.77      0.72      2223



In [43]:
#Using Word2Vec

In [44]:
# import gensim
# model = gensim.models.Word2Vec() 
# GoogleModel = gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\SHWETANK VERMA\Documents\Mlstuff\Major-1\GoogleNews-vectors-negative300.bin', binary=True,)


In [45]:
# Split data into train and test sets
import gensim
X_train, X_test, y_train, y_test = train_test_split (train['Text_data'], train['Label'] , test_size=0.2)
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)
    

In [46]:
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])

In [47]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [34]:
# Decision tree with gini
model_gini = DecisionTreeClassifier(criterion = "gini",
            random_state = 123,max_depth=10, min_samples_leaf=6)
  
# Performing training
model_gini.fit(X_train_vect_avg, y_train.values.ravel())

DecisionTreeClassifier(max_depth=10, min_samples_leaf=6, random_state=123)

In [35]:
len(X_test_vect_avg)

1779

In [36]:

# Prediction using gini
y_pred = prediction(X_test_vect_avg, model_gini)
cal_accuracy(y_test,y_pred)
print(classification_report(y_test,y_pred))

ValueError: X has 12150 features, but DecisionTreeClassifier is expecting 100 features as input.